# Chapter 2. 파이썬스러운(pythonic) 코드
+ 이 장의 목표
    + 인덱스와 슬라이스를 이해하고 인덱싱 가능한 객체를 올바른 방식으로 구현
    + 시퀀스와 이터러블 구현
    + 매직 메서드를 사요해 보다 관용적인 코드 구현
    + 부작용을 유발하는 흔한 실수 피하기

## 인덱스와 슬라이스

In [12]:
my_numbers = (4, 5, 3, 9)

print("음수 인덱스:", my_numbers[-1], my_numbers[-3])
print("slice 1:", my_numbers[2:5])
print("slice 2:", my_numbers[:3])
print("slice 3:", my_numbers[3:])
print("slice 4:", my_numbers[::-1])

print("\n")
print("slice index:", my_numbers[1:7:2])
print("slice object:", my_numbers[slice(1, 7, 2)])

print("\n")
interval = slice(None, 3)
print("slice object vs index:", my_numbers[interval] == my_numbers[:3])

음수 인덱스: 9 5
slice 1: (3, 9)
slice 2: (4, 5, 3)
slice 3: (9,)
slice 4: (9, 3, 5, 4)


slice index: (5, 9)
slice object: (5, 9)


slice object vs index: True


## 자체 시퀀스 생성
+ 범위로 인덱싱하는 결과는 해당 클래스와 같은 타입의 인스턴스여아 한다.
+ slice에 의해 제공된 범위는 파이썬이 하는 것처럼 마지막 요소는 제외한다.

In [35]:
""" list wrapping 사례 """

class Item:
    def __init__(self, *values):
        self._value = list(values)
    
    def __len__(self):
        return len(self._value)

    def __getitem__(self, index):
        return self._value.__getitem__(index)

In [36]:
item = Item(3, 4, 5)
print(len(item))
print(item[1:])
print(type(item[1:]))

3
[4, 5]
<class 'list'>


## 컨텍스트 관리자
+ __enter__와 __exit__ 두 개의 매직 메서드로 구성됨
+ with 문은 __enter__ 메서드를 호추랗고 이 메서드가 무엇을 반환하든 as 이후에 지정된 변수에 할당됨
+ 해당 블록에 대한 마지막 문장이 끝나면 컨텍스트가 종료되며 __exit__ 메서드를 호출함
+ 블록 내에 예외 또는 오류가 있는 경우에도 __exit__ 메서드가 여전히 호출되며, 예외는 파라미터로 확인 가능
    + True를 반환하면 잠재적으로 발생한 예외를 호출자에게 전파하지 않고 정지함을 의미하나 좋지 않은 습관임